In [1]:
import sys.process._
import java.net.URLDecoder
import scala.util.{Try, Success, Failure}
import org.apache.spark.sql.functions.udf
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.catalyst.ScalaReflection
case class Schemalog(
    UID: String,
    timestamp: String,
    URL: String
)
val schemaFromClass = ScalaReflection.schemaFor[Schemalog].dataType.asInstanceOf[StructType]
def parser(url: String) = {
    Try {new java.net.URL(URLDecoder.decode(url, "UTF-8"))}.toOption
      match {
        case Some(_) => new java.net.URL(URLDecoder.decode(url, "UTF-8")).getHost
        case _ => "unable_to_parse"
      }
    }

defined class Schemalog
schemaFromClass = StructType(StructField(UID,StringType,true), StructField(timestamp,StringType,true), StructField(URL,StringType,true))


parser: (url: String)String


StructType(StructField(UID,StringType,true), StructField(timestamp,StringType,true), StructField(URL,StringType,true))

In [2]:
val autom = spark.read.option("header", "true").option("multiline", "true").option("encoding", "UTF-8").json("/labs/laba02")

Waiting for a Spark session to start...

autom = [autousers: array<string>]


[autousers: array<string>]

In [3]:
val autom_explode = autom.select(org.apache.spark.sql.functions.explode($"autousers").alias("autousers"))

autom_explode = [autousers: string]


[autousers: string]

In [4]:
val csvOptions = Map("header" -> "true", "inferSchema" -> "false", "sep" -> "\t", "encoding" -> "UTF-8", "charset" -> "UTF8" )
val log = spark.read.options(csvOptions).schema(schemaFromClass).csv("/labs/laba02/logs/")

csvOptions = Map(encoding -> UTF-8, inferSchema -> false, sep -> "	", charset -> UTF8, header -> true)
log = [UID: string, timestamp: string ... 1 more field]


[UID: string, timestamp: string ... 1 more field]

In [5]:
val log_auto=log.join(autom_explode, log("UID") ===  autom_explode("autousers"), "left")


log_auto = [UID: string, timestamp: string ... 2 more fields]


[UID: string, timestamp: string ... 2 more fields]

In [6]:

val isAlienNameUDF2 = udf(parser _)
val df5 = log_auto.filter(col("URL").like("http%") and col("URL").isNotNull and !col("URL").like("")).withColumn("df5", 
isAlienNameUDF2(col("URL")))

isAlienNameUDF2 = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))
df5 = [UID: string, timestamp: string ... 3 more fields]


[UID: string, timestamp: string ... 3 more fields]

In [7]:
import org.apache.spark.sql.functions.udf

val isAlienNameUDF2 = udf(parser _)

isAlienNameUDF2 = UserDefinedFunction(<function1>,StringType,Some(List(StringType)))


UserDefinedFunction(<function1>,StringType,Some(List(StringType)))

In [8]:
parser("http%3A%2F%2Favto-russia.ru%2Fautos%2F")

avto-russia.ru

In [9]:
val df5 = log_auto.filter(col("URL").like("http%") and col("URL").isNotNull and !col("URL").like("")).withColumn("df5", 
isAlienNameUDF2(col("URL")))

df5 = [UID: string, timestamp: string ... 3 more fields]


[UID: string, timestamp: string ... 3 more fields]

In [10]:
val log_auto_fil_2 = df5.filter(col("df5").like("www.%")).withColumn("ur", regexp_replace($"df5", "www.", "")).union(df5.filter(!col("df5").like("www.%")).withColumn("ur", col("df5")))

log_auto_fil_2 = [UID: string, timestamp: string ... 4 more fields]


[UID: string, timestamp: string ... 4 more fields]

In [11]:
val log_auto_flag = log_auto_fil_2.withColumn("flag", when(col("autousers").isNull , 0).otherwise(1))
val cnt = log_auto_flag.count()
val cnt_1 = log_auto_flag.filter(col("flag") === 1).count()
val log_auto_1 = log_auto_flag.groupBy("ur").agg(sum("flag").alias("gr_1"), count("flag").alias("gr_all"))

log_auto_flag = [UID: string, timestamp: string ... 5 more fields]
cnt = 6570395
cnt_1 = 313498
log_auto_1 = [ur: string, gr_1: bigint ... 1 more field]


[ur: string, gr_1: bigint ... 1 more field]

In [12]:
val log_auto_rel = log_auto_1.withColumn("relevance", format_number((col("gr_1")/lit(cnt))*(col("gr_1")/lit(cnt))/((col("gr_all")/lit(cnt)) * (lit(cnt_1)/lit(cnt))), 15) )

log_auto_rel = [ur: string, gr_1: bigint ... 2 more fields]


[ur: string, gr_1: bigint ... 2 more fields]

In [13]:
log_auto_rel.orderBy(col("relevance").desc, col("ur") ).select($"ur",$"relevance").limit(200).repartition(1).write.mode("overwrite").option("header",false).option("encoding", "UTF-8").option("sep","\t").csv("laba02_5_domains.txt")

"""hdfs dfs -get laba02_5_domains.txt""".!!